<a href="https://colab.research.google.com/github/AleksanderSokolov/algorithm/blob/master/ForestTreeProgect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Случайный лес

In [0]:
N_TREES=250
MIN_LEAF_FROM=1
MIN_LEAF_TO=6
TEST_SIZE=0.1

import time
start_time = time.time()

### Реализация дерева решений

In [0]:
import matplotlib.pyplot as plt
import random

from matplotlib.colors import ListedColormap
from sklearn import datasets

import numpy as np

In [0]:
import pandas as pd

In [43]:
# Info on how to get your api key (kaggle.json) here: https://github.com/Kaggle/kaggle-api#api-credentials
!pip install kaggle
api_token = {"username":"sokolovaleksander","key":"23f64fa7ccc4dc7e849121b0ea598aae"}
import json
import zipfile
import os
from google.colab import drive
drive.mount('/content/drive')
# copy kaggle token file from google drive to colab
!mkdir -p ~/.kaggle
!cp "/content/drive/My Drive/Colab Notebooks/kaggle/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle
!pip install kaggle

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
kaggle.json


In [44]:
!kaggle competitions download -c choose-tutors

test.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv: Skipping, found more recently modified local copy (use --force to force download)
submission_example.csv: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
df = pd.read_csv('train.csv')

In [0]:
dfTest = pd.read_csv('test.csv')

In [47]:
df.describe()

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points,choose
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,4999.50000,45.800900,1.974800,1702.440000,1.724300,0.37060,0.121500,0.117200,0.059100,0.02770,0.018000,64.435200,0.110900
std,2886.89568,8.030274,1.766883,523.789062,0.798845,0.48299,0.326724,0.321675,0.235824,0.16412,0.132958,13.595024,0.314024
min,0.00000,23.000000,0.000000,200.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,33.000000,0.000000
25%,2499.75000,40.000000,0.000000,1300.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,55.000000,0.000000
50%,4999.50000,46.000000,2.000000,1550.000000,2.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,63.000000,0.000000
75%,7499.25000,51.000000,3.000000,2150.000000,2.000000,1.00000,0.000000,0.000000,0.000000,0.00000,0.000000,74.000000,0.000000
max,9999.00000,68.000000,9.000000,3950.000000,4.000000,1.00000,1.000000,1.000000,1.000000,1.00000,1.000000,100.000000,1.000000


In [0]:
arrChoose = df['choose'].values

In [0]:
arr = df.drop(columns=['choose','Id'], axis=1).values

In [50]:
df.columns.values

array(['Id', 'age', 'years_of_experience', 'lesson_price',
       'qualification', 'physics', 'chemistry', 'biology', 'english',
       'geography', 'history', 'mean_exam_points', 'choose'], dtype=object)

In [0]:
# Разобьем выборку на обучающую и тестовую

from sklearn import model_selection

train_data, test_data, train_labels, test_labels = model_selection.train_test_split(arr, 
                                                                                     arrChoose, 
                                                                                     test_size = TEST_SIZE,
                                                                                     random_state = 1)

In [0]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [0]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        #classes = {}  # сформируем словарь "класс: количество объектов"
        classes = {0: 0, 1: 0}
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)

        pred_proba = []
        for label in classes:
            p = classes[label] / len(self.labels)
            pred_proba.append(p)

        return prediction, pred_proba

In [0]:
# Расчет критерия Джини

def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [0]:
# Расчет качества

def quality(left_labels, right_labels, current_gini):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

In [0]:
# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [0]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels,min_leaf):
    
    #  обозначим минимальное количество объектов в узле
    #min_leaf = 0.9

    current_gini = gini(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее min_leaf объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_gini)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [0]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels, min_leaf):

    quality, t, index = find_best_split(data, labels,min_leaf)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels, min_leaf)
    false_branch = build_tree(false_data, false_labels, min_leaf)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

Формирование случайного леса


Реализуем генерацию  N  бутстрап-выборок и подмножества признаков для нахождения разбиения в узле ??

In [0]:
random.seed(42)

def get_bootstrap(data, labels, N):
    n_samples = data.shape[0]
    bootstrap = []
    
    for i in range(N):
        b_data = np.zeros(data.shape)
        b_labels = np.zeros(labels.shape)
        
        for j in range(n_samples):
            sample_index = random.randint(0, n_samples-1)
            b_data[j] = data[sample_index]
            b_labels[j] = labels[sample_index]
        bootstrap.append((b_data, b_labels))
        
    return bootstrap

In [0]:
random.seed(42)
def random_forest(data, labels, n_trees):
    forest = []
    bootstrap = get_bootstrap(data, labels, n_trees)
    
    for b_data, b_labels in bootstrap:
        min_leaf = random.randint(MIN_LEAF_FROM, MIN_LEAF_TO)
        forest.append(build_tree(b_data, b_labels, min_leaf))
        
    return forest

In [0]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer, pred_proba  = node.prediction
        return answer, pred_proba 
 
    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [0]:
def predict(data, tree):
    
    classes = []
    probability = []
    for obj in data:
        prediction, pred_proba = classify_object(obj, tree)
        classes.append(prediction)
        probability.append(pred_proba)
    probability = np.array(probability)
    classes = np.array(classes)
    return classes, probability

In [0]:
# предсказание голосованием деревьев

def tree_vote(forest, data):

    # добавим предсказания всех деревьев в список
    predictions = []
    pred_proba = []
    for tree in forest:
        classes, probability = predict(data, tree)
        predictions.append(classes)
        pred_proba.append(probability)
    
    # сформируем список с предсказаниями для каждого объекта
    predictions_per_object = list(zip(*predictions))
    proba_per_object = list(zip(*pred_proba))
    
    # выберем в качестве итогового предсказания для каждого объекта то,
    # за которое проголосовало большинство деревьев
    voted_predictions = []
    for obj in predictions_per_object:
        voted_predictions.append(max(set(obj), key=obj.count))
        
    voted_proba = []
    for obj in proba_per_object:
        sum_proba_of_1 = 0
        sum_proba_of_2 = 0
        sum_proba_of_3 = 0
        sum_proba_of_4 = 0

        i=0
        k=0
        l=0
        m=0        
        for p in obj:
            if p[1]>0.75:
                sum_proba_of_1 += p[1]
                i+=1
            elif p[1]>0.5:                                   
                sum_proba_of_2 += p[1]
                l+=1            
            elif p[1]>0.25:
                k+=1
                sum_proba_of_3 += p[1]
            elif p[1]>=0:
                m+=1
                sum_proba_of_4 += p[1]

        if i>k and i>l and i>m:
          sum_proba_of=(sum_proba_of_1+sum_proba_of_2+sum_proba_of_3)/(i+k+l)
          #sum_proba_of=(sum_proba_of_1+sum_proba_of_2)/(i+l)
        elif l>k and l>i and l>m:
          sum_proba_of=(sum_proba_of_1 + sum_proba_of_2+sum_proba_of_3)/(l+k+i)
          #sum_proba_of=(sum_proba_of_1 + sum_proba_of_2)/(l+i)
        elif k>i and k>l and k>m:
          sum_proba_of=(sum_proba_of_2+sum_proba_of_3+sum_proba_of_4)/(k+l+m)
          #sum_proba_of=(sum_proba_of_3+sum_proba_of_4)/(k+m)
        elif m>i and m>l and m>k:
          sum_proba_of=(sum_proba_of_3+sum_proba_of_4+sum_proba_of_2)/(k+m+l)
          #sum_proba_of=(sum_proba_of_4+sum_proba_of_3)/(m+k)
        else:
          sum_proba_of=(sum_proba_of_1 + sum_proba_of_2+sum_proba_of_3+sum_proba_of_4)/(i+l+k+m)

        voted_proba.append(sum_proba_of)
        
    voted_predictions = np.array(voted_predictions)
    voted_proba = np.array(voted_proba)
        
    return voted_predictions, voted_proba

Построим лес

In [64]:
%%time
n_trees = N_TREES
my_forest = random_forest(train_data, train_labels, n_trees)

CPU times: user 22min 58s, sys: 13.4 s, total: 23min 11s
Wall time: 23min 12s


In [0]:
# Получим ответы для обучающей выборки 
train_answers, train_proba = tree_vote(my_forest, train_data)

In [0]:
# И получим ответы для тестовой выборки
test_answers, test_proba = tree_vote(my_forest, test_data)

In [0]:
# Введем функцию подсчета точности как доли правильных ответов
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [68]:
# Точность на обучающей выборке
train_accuracy = accuracy_metric(train_labels, train_answers)
print(f'Точность случайного леса из {n_trees} деревьев на обучающей выборке: {train_accuracy:.3f}')

Точность случайного леса из 250 деревьев на обучающей выборке: 94.911


In [69]:
# Точность на тестовой выборке
test_accuracy = accuracy_metric(test_labels, test_answers)
print(f'Точность случайного леса из {n_trees} деревьев на тестовой выборке: {test_accuracy:.3f}')

Точность случайного леса из 250 деревьев на тестовой выборке: 89.500


In [70]:
import numpy as np
from sklearn.metrics import roc_auc_score
roc_auc_test=roc_auc_score(test_labels, test_proba)
roc_auc_test

0.7858303373893805

In [71]:
roc_auc_score(train_labels, train_proba)

0.9960583776870385

In [0]:
# И получим ответы для всей выборки
all_answers, all_proba  = tree_vote(my_forest, dfTest.drop(columns=['Id'], axis=1).values)

In [0]:
dfTest['choose']=all_proba

In [74]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
dfTest.loc[:, ['Id', 'choose']].to_csv('submission.csv',index=False, encoding='utf-8')

In [0]:
message="roc_auc_test ="+str(roc_auc_test)+"n_tr ="+str(N_TREES)+", min_leaf "+str(MIN_LEAF_FROM)+"-"+str(MIN_LEAF_TO)+", test "+str(TEST_SIZE) + " time:" +str(time.time()-start_time)

In [77]:
message

'roc_auc_test =0.7858303373893805n_tr =250, min_leaf 1-6, test 0.1 time:1453.1408338546753'

In [78]:
if roc_auc_test>0.79:
  !kaggle competitions submit -c choose-tutors -f submission.csv -m '$message'
else:
   print("мало")



мало
